# Exploring UC1004 dataset

In [ ]:
import sys
sys.path.insert(0, './utils/')

import geo_conv
import sqlite3 as sqlite
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium

plt.style.use('ggplot')
%matplotlib inline

Database schema: network, us_pos, us_signal

In [ ]:
def add_lat_lon(df):
    xy = zip(df.x.values, df.y.values)
    latlon = map(lambda (coord):geo_conv.convert_GK_to_lat_long(coord[0]+__XMIN, coord[1]+__YMIN), xy)
    
    if 'lat' not in df.columns:
        df = pd.concat([df, 
                        pd.DataFrame.from_records(latlon, columns=['lat', 'lon'])],
                       axis=1)
    else:
        df[['lat', 'lon']] = pd.DataFrame.from_records(latlon, columns=['lat', 'lon'])
        
    return df

In [ ]:
__CITY_LAT = 52.3667
__CITY_LON = 9.7167
__XMIN = 4336000
__YMIN = 5794000

In [ ]:
# Setting conntection to sqlite db
conn = sqlite.connect('./data/uc1004/uc1004.db')

## Getting Networkd data

In [ ]:
# Get Network data file
query = 'SELECT * FROM network;'

network = pd.read_sql(query, conn)
network.shape

In [ ]:
network = add_lat_lon(network)
network.set_index('cell_id', inplace=True)
network.head(10)

## Getting User Position

As it goes from the article, users are categorized to:<br\>
Vehicular: 0 ... 4619 <br\>
Pedestrian: 10000 ... 15246<br\>
Static Indoor: 20000 ... 29999

In [ ]:
user_id = 10003
query = 'SELECT * FROM us_pos WHERE userid=%d' % user_id
user_pos = pd.read_sql(query, conn)

In [ ]:
print user_pos.shape
user_pos.sort_values('time', inplace=True)
user_pos.head()

In [ ]:
# Adding lat, long columns
user_pos = add_lat_lon(user_pos)
user_pos.head()

## Getting User's RSPS

In [ ]:
query = 'SELECT * FROM us_signal WHERE userid=%d' % user_id
user_signals = pd.read_sql(query, conn)

In [ ]:
print user_signals.shape
user_signals.sort_values('time', inplace=True)
user_signals.head()

In [ ]:
# Identify cell set for user
cell_id_cols = range(2, 42, 2)
cell_ids = np.unique(user_signals.iloc[:, cell_id_cols].values.astype(int))

## Plotting Maps From Dataset

### Land-case map

In [ ]:
# Land-case map
use_map = np.genfromtxt('Land_Use_Map.csv', delimiter=';')

In [ ]:
print use_map.shape # It differes from info in the article
print np.unique(use_map)

In [ ]:
# .T to have refference with pics in the article
# But map shoulb be rotated by 90 degrees (according to google maps)
# Moreover, the map in data is not the same as in the article..
# fig = plt.figure(figsize=(10, 20))
# plt.imshow(use_map.T, cmap='flag')

In [ ]:
use_map = np.rot90(use_map)

In [ ]:
fig = plt.figure(figsize=(10, 20))
plt.imshow(use_map, cmap='flag')

### Best server map

In [ ]:
best_server = np.genfromtxt('Best_Server_Map.csv', delimiter=';')

In [ ]:
print best_server.shape # It differes from info in the article
print np.unique(best_server) # should be at least = to # of cells

In [ ]:
best_server = np.rot90(best_server)

In [ ]:
fig = plt.figure(figsize=(10, 20))
plt.imshow(best_server, cmap='flag')

## Playing with real map

In [ ]:
map_hannover = folium.Map(location=[__CITY_LAT, __CITY_LON], 
                          zoom_start=15)
map_hannover.line(locations=user_pos[['lat', 'lon']].values,
                  line_opacity=0.5, 
                  popup=('user: ' + str(user_id)))
for cid in cell_ids:
    cell = network.loc[cid]
    map_hannover.polygon_marker(location=[cell.lat, cell.lon],
                                num_sides=3,
                                radius=5,
                                rotation=cell.azimuth,
                                popup='cell_id: ' + str(cid) + '\nazimuth: ' + str(cell.azimuth))

map_hannover.create_map(path='./data/uc1004/map.html')

## Making user signals in better way

In [ ]:
signals_long = (pd.wide_to_long(user_signals, ['cell_id', 'rsr'], i='time', j='cell_order')
                  .reset_index()
                  .set_index(['cell_id', 'time'])
                  .sort_index(level=0))

In [ ]:
cell_id = 6
signals_long.loc[cell_id].plot(y='rsr'
                              )
fig = plt.gcf()
fig.set_size_inches(20,10)
plt.ylabel('RSR')
plt.xlabel('time')